## Lending Club Case Study

### Setting up the base and dividing each section into the steps performed. 
#### We will perform the below task on the data set and provide the result
1. We will get the data from the file (As data is provided in csv file).
2. Look at the data set and perform the EDA on it.
3. Exclude the column which are not relevant nd mentione the column which are to be used.
4. Provide the visualization on the data to support the result.

In [26]:
#Loading all the necessary library for usage
#Imports - Starts
import pandas as pd     #Importing Panda's Library 
import numpy as np
import seaborn as sb
import matplotlib as mpl
import math as math
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
#Imports - End

#Load Data from the data set file. It requires the file to be present in the local system 

raw_dataset = pd.read_csv("/Users/sing73/upgrad/ACP/Data_LendingClubCaseStudy/loan.csv")

# Getting the information of the data frame
raw_dataset.info()
#Looking at 1st 10 rows of the data frame 
raw_dataset.head(10)

# Looking at all the records in the data frame
#raw_dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


/Users/sing73/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.



,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
5,1075269,1311441,5000,5000,5000.0,36 months,7.90%,156.46,A,A4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
6,1069639,1304742,7000,7000,7000.0,60 months,15.96%,170.08,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
7,1072053,1288686,3000,3000,3000.0,36 months,18.64%,109.43,E,E1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
8,1071795,1306957,5600,5600,5600.0,60 months,21.28%,152.39,F,F2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
9,1071570,1306721,5375,5375,5350.0,60 months,12.69%,121.45,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [27]:
# Function will be defined below which we have to use during the data processing
# Function Def - Starts 

# - We need to define the employee service duration in number for usage
def employee_service_duration_to_number(emp_yr):
    f_no = emp_yr.split(" ")[0].strip()
    if('<' in emp_yr ):
        return 0
    if ('+' in f_no):
        return f_no.split("+")[0].strip()
    else:
        return f_no

# Function Def - End


##### There are 111 column and 39717 rows in the data set.
##### Keeping all the row will not be feasible as to derive the result hence there will be some limited coloum which will be used out of 111. Possibly 20 Column out of 111
##### The data is the loan data hence 
##### For EDA we will not consider the column having more than 80% of data as empty data 


In [40]:

# Getting the list of the records for column that are relevant for us
filtered_raw_data = raw_dataset[['loan_amnt','term','int_rate','installment','grade','sub_grade','emp_length','home_ownership','annual_inc',
                                 'verification_status','loan_status','dti','purpose', 'acc_now_delinq','addr_state',
                                 'pub_rec_bankruptcies','funded_amnt','funded_amnt_inv','title','mths_since_last_delinq']]

# We need to get the data for the loan_status where loan_status is Charged Off / Fully Paid and Current we cannot determine whether it will be defaulter or not
loan_status_data_set = filtered_raw_data [(filtered_raw_data['loan_status'].isin(['Charged Off','Fully Paid']))]


#Remove the % and from int_rate and  months from term 
loan_status_data_set['int_rate'] = loan_status_data_set['int_rate'].astype(str).str.replace("%", "").astype(float)
loan_status_data_set['term'] = loan_status_data_set['term'].astype(str).str.replace("months", "").astype(int)

loan_status_data_set.isnull().sum()




,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,...,purpose,acc_now_delinq,addr_state,pub_rec_bankruptcies,funded_amnt,funded_amnt_inv,title,mths_since_last_delinq,bin_int_rate,bin_installment
0,5000,36,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,...,credit_card,0,AZ,0.0,5000,4975.0,Computer,NaN,"(10, 15]","(100.0, 200.0]"
1,2500,60,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,...,car,0,GA,0.0,2500,2500.0,bike,NaN,"(15, 20]","(0.0, 100.0]"
2,2400,36,15.96,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,...,small_business,0,IL,0.0,2400,2400.0,real estate business,NaN,"(15, 20]","(0.0, 100.0]"
3,10000,36,13.49,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,...,other,0,CA,0.0,10000,10000.0,personel,35.0,"(10, 15]","(300.0, 400.0]"
5,5000,36,7.90,156.46,A,A4,3 years,RENT,36000.0,Source Verified,...,wedding,0,AZ,0.0,5000,5000.0,My wedding loan I promise to pay back,NaN,"(5, 10]","(100.0, 200.0]"
6,7000,60,15.96,170.08,C,C5,8 years,RENT,47004.0,Not Verified,...,debt_consolidation,0,NC,0.0,7000,7000.0,Loan,NaN,"(15, 20]","(100.0, 200.0]"
7,3000,36,18.64,109.43,E,E1,9 years,RENT,48000.0,Source Verified,...,car,0,CA,0.0,3000,3000.0,Car Downpayment,NaN,"(15, 20]","(100.0, 200.0]"
8,5600,60,21.28,152.39,F,F2,4 years,OWN,40000.0,Source Verified,...,small_business,0,CA,0.0,5600,5600.0,Expand Business & Buy Debt Portfolio,NaN,"(20, 25]","(100.0, 200.0]"
9,5375,60,12.69,121.45,B,B5,< 1 year,RENT,15000.0,Verified,...,other,0,TX,0.0,5375,5350.0,Building my credit history.,NaN,"(10, 15]","(100.0, 200.0]"
10,6500,60,14.65,153.45,C,C3,5 years,OWN,72000.0,Not Verified,...,debt_consolidation,0,AZ,0.0,6500,6500.0,High intrest Consolidation,NaN,"(10, 15]","(100.0, 200.0]"


In [ ]:

# Making the groupings for the interest rate
bins = [0,5,10,15,20,25]
loan_status_data_set['bin_int_rate'] = pd.cut(loan_status_data_set['int_rate'], bins)
loan_status_data_set['bin_int_rate'].value_counts()

# Making the groupings for the installment
bins_installment = np.linspace(0,1400,15)
loan_status_data_set['bin_installment'] = pd.cut(loan_status_data_set['installment'], bins_installment)
loan_status_data_set['bin_installment'].value_counts()

In [29]:
loan_status_data_set.loan_status.unique() # check if the loan status is only 'Fully Paid' and 'Charged Off'

array(['Fully Paid', 'Charged Off'], dtype=object)

In [34]:
#Since we have obtained the loan_status data we can update the data of interest and employee length. Interest contain % sign as non numeric value 
# and employee length has string so convert in  numbers for usage

loan_status_data_set['emp_length'] = loan_status_data_set['emp_length'].astype(str).apply(lambda x: employee_service_duration_to_number(x))

# replacing all NaN with 0 to avoid any error
loan_status_data_set['emp_length'] = loan_status_data_set['emp_length'].str.replace("nan","0") # replace the emp_length as Mode 

#Convert the emp_length as numeric value 
loan_status_data_set['emp_length'] = pd.to_numeric(loan_status_data_set['emp_length'])

#Verify the emp_length if it is only numeric values
loan_status_data_set.emp_length.unique()

loan_status_data_set

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,dti,purpose,acc_now_delinq,addr_state,pub_rec_bankruptcies,funded_amnt,funded_amnt_inv,title,mths_since_last_delinq
0,5000,36,10.65,162.87,B,B2,10.0,RENT,24000.0,Verified,Fully Paid,27.65,credit_card,0,AZ,0.0,5000,4975.0,Computer,NaN
1,2500,60,15.27,59.83,C,C4,0.0,RENT,30000.0,Source Verified,Charged Off,1.00,car,0,GA,0.0,2500,2500.0,bike,NaN
2,2400,36,15.96,84.33,C,C5,10.0,RENT,12252.0,Not Verified,Fully Paid,8.72,small_business,0,IL,0.0,2400,2400.0,real estate business,NaN
3,10000,36,13.49,339.31,C,C1,10.0,RENT,49200.0,Source Verified,Fully Paid,20.00,other,0,CA,0.0,10000,10000.0,personel,35.0
5,5000,36,7.90,156.46,A,A4,3.0,RENT,36000.0,Source Verified,Fully Paid,11.20,wedding,0,AZ,0.0,5000,5000.0,My wedding loan I promise to pay back,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,2500,36,8.07,78.42,A,A4,4.0,MORTGAGE,110000.0,Not Verified,Fully Paid,11.33,home_improvement,0,CO,NaN,2500,1075.0,Home Improvement,0.0
39713,8500,36,10.28,275.38,C,C1,3.0,RENT,18000.0,Not Verified,Fully Paid,6.40,credit_card,0,NC,NaN,8500,875.0,Retiring credit card debt,5.0
39714,5000,36,8.07,156.84,A,A4,0.0,MORTGAGE,100000.0,Not Verified,Fully Paid,2.30,debt_consolidation,0,MA,NaN,5000,1325.0,MBA Loan Consolidation,0.0
39715,5000,36,7.43,155.38,A,A2,0.0,MORTGAGE,200000.0,Not Verified,Fully Paid,3.72,other,0,MD,NaN,5000,650.0,JAL Loan,0.0


In [35]:
loan_status_data_set.isnull().sum()

loan_amnt                     0
term                          0
int_rate                      0
installment                   0
grade                         0
sub_grade                     0
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
loan_status                   0
dti                           0
purpose                       0
acc_now_delinq                0
addr_state                    0
pub_rec_bankruptcies        697
funded_amnt                   0
funded_amnt_inv               0
title                        11
mths_since_last_delinq    24905
dtype: int64

In [32]:
#labels
lab = loan_status_data_set["verification_status"].value_counts().keys().tolist()
#values
val = loan_status_data_set["verification_status"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime', 'yellow'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Customer Verification Data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)

In [33]:
#labels
lab = loan_status_data_set["home_ownership"].value_counts().keys().tolist()
#values
val = loan_status_data_set["home_ownership"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime', 'yellow', 'red', 'pink'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Home Ownership Data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)